In [1]:
%load_ext autoreload
%autoreload 2

from data.visualise import scatter_3d, show_psf_axial
from data.datasets import TrainingDataSet, ExperimentalDataSet
from config.datasets import dataset_configs
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from debug_tools.view_dataset_psfs import grid_psfs
import numpy as np
import matplotlib.pyplot as plt

In [8]:

dataset_name = 'openframe'
sub_name = 'experimental'
z_range = 1000
cfg = dataset_configs[dataset_name][sub_name]


dataset = ExperimentalDataSet(cfg, z_range, transform_data=False, lazy=True)
dataset.filter_emitters_proximity = False
dataset.prepare_data()
all_psfs = dataset.data[0]


# dataset = TrainingDataSet(cfg, z_range, lazy=True, transform_data=False, normalize_psf=False, filter_emitters_proximity=True, disable_emitter_peak_boundary=True)
dataset.csv_backup = dataset.csv_data.copy()

print(all_psfs.shape)
# def norm_peak_layer(psf):
#     peak_idx = np.argmax(psf.max(axis=(1,2)))
#     psf = psf[[peak_idx]]
#     psf = psf / psf.max()
#     return psf
# all_psfs = np.stack([norm_peak_layer(psf) for psf in all_psfs])
psf_idx = dataset.kept_idx


 47%|████▋     | 147/316 [00:00<00:00, 1460.57it/s]

Reading img...
Loading /home/miguel/Projects/uni/data/smlm_3d/bead_3D_STORM_three_instruments/20200723_OpenFrame_3D_beads/glass_beads_1mm_agarose_center_1/glass_beads_1mm_agarose_center_1_MMStack_Default_substack.ome.tif
Loading /home/miguel/Projects/uni/data/smlm_3d/bead_3D_STORM_three_instruments/20200723_OpenFrame_3D_beads/glass_beads_1mm_agarose_center_1/glass_beads_1mm_agarose_center_1_MMStack_Default_substack.csv
334 emitters before filtering
316 emitters after borders


100%|██████████| 316/316 [00:00<00:00, 1467.51it/s]

(316, 32, 32, 1)


In [9]:
print(all_psfs.shape)

(316, 32, 32, 1)


In [10]:

selected_psfs = None
def filter_csv_and_display(*args, **kwargs):
    global all_psfs
    global selected_psfs
    print(f'{dataset.csv_backup.shape[0]} PSFs found in img.')
    csv_data = dataset.csv_backup.copy()
    for col_name, vals in kwargs.items():
        n_before = csv_data.shape[0]
        csv_data = csv_data[(csv_data[col_name] >= vals[0]) & (csv_data[col_name] <= vals[1])]
        n_after = csv_data.shape[0]
        print(f'\t{n_before} -> {col_name} -> {n_after}')

    selected_idx = [psf_idx.index(i) for i in csv_data.index if i in psf_idx]
    psfs = [all_psfs[i] for i in selected_idx]
    plt.rcParams['figure.figsize'] = [3, 3]

    selected_psfs = np.array(psfs)
    plt.rcParams['figure.figsize'] = [30, 50]
    print(selected_psfs.shape)
    psfs = grid_psfs(np.moveaxis(selected_psfs, 3, 1), cols=30).squeeze()
    plt.imshow(psfs)


In [11]:
base_csv = dataset.csv_backup
cols = {}
for col_name in sorted(list(base_csv)):
    if col_name in ['frame', 'x [nm]', 'y [nm]']:
        continue
    widget = widgets.FloatRangeSlider(min=base_csv[col_name].min(), max=base_csv[col_name].max(), value=[0, base_csv[col_name].max()])
    cols[col_name] = widget
    w = interactive(filter_csv_and_display, {'manual': base_csv.shape[0] > 1000}, **cols)
display(w)

interactive(children=(FloatRangeSlider(value=(22.50232, 182.4363), description='bkgstd [photon]', max=182.4363…

In [6]:
eg_psf = psfs[0]

import numpy as np


def fit_gaussian(psf):
    def twoD_Gaussian(coords, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
        (x, y) = coords
        xo = float(xo)
        yo = float(yo)    
        a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
        b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
        c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
        g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                                + c*((y-yo)**2)))
        return g.ravel()

    initial_amp = psf.max()
    initial_x0 = psf.shape[-1]//2
    initial_y0 = psf.shape[-2]//2
    sigma_x = initial_x0/10
    sigma_y = initial_y0/10
    theta = 1
    offset = psf.max() - psf.min()
    
    initial_guess = (initial_amp, initial_x0, initial_y0, sigma_x, sigma_y, theta, offset)

#     data_noisy = data + 0.2*np.random.normal(size=data.shape)

    popt, pcov = opt.curve_fit(twoD_Gaussian, (x, y), psf, p0=initial_guess)

fit_gaussian(eg_psf)

NameError: name 'psfs' is not defined